# Create ML RandomForest Model

In [1]:
#Import dependencies
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from config import db_password

In [2]:
#Make a connection to Postgres to load in the dataframe
connection = psycopg2.connect(user="postgres",
                                  password=db_password,
                                  host="localhost",
                                  port="5432",
                                  database="Netflix")
postgre = "select * from netflix_data_final"
netflix_copy = pd.read_sql_query(postgre, connection)
#Close connection
connection = None

In [4]:
X = netflix_copy.copy()
X = X.drop(["Rating", "index"], axis=1)
X.head(10)

,Title,Start Year,Run Time,Type,Origin Country,Language,Number of Votes,Release Year,Movie Rating,Comedy,...,Mystery,Romance,Music,Fantasy,Musical,Talk-Show,Sci-Fi,Sport,History,Other
0,1052.0,2016,42.0,1.0,57.0,10.0,250884,2021,2.0,0,...,0,0,0,1,0,0,0,0,0,0
1,148.0,2021,148.0,0.0,57.0,10.0,110780,2021,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1864.0,2018,30.0,1.0,57.0,10.0,28795,2021,0.0,1,...,0,0,0,0,0,0,0,0,0,1
3,1390.0,2020,45.0,1.0,38.0,35.0,26606,2021,0.0,0,...,0,0,0,1,0,0,0,0,0,0
4,1636.0,2016,44.0,1.0,57.0,10.0,16980,2018,0.0,0,...,0,0,0,0,0,0,0,0,0,1
5,704.0,2005,41.0,1.0,57.0,10.0,260703,2020,2.0,0,...,0,1,0,0,0,0,0,0,0,1
6,1739.0,2013,43.0,1.0,57.0,10.0,207174,2019,2.0,0,...,1,0,0,0,0,0,0,0,0,0
7,925.0,2021,56.0,1.0,57.0,10.0,27309,2021,0.0,0,...,0,0,0,0,0,0,0,0,0,0
8,501.0,2018,44.0,1.0,57.0,10.0,16578,2020,0.0,0,...,0,0,0,0,0,0,0,0,0,1
9,1982.0,2010,44.0,1.0,57.0,10.0,873752,2019,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y = netflix_copy['Rating'].values
y[:5]

array([0, 1, 0, 0, 0], dtype=int64)

In [6]:
# Train testing groups
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1486, 32)
(496, 32)
(1486,)
(496,)


In [8]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=1)

In [10]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [11]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,

In [12]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,152,66
Actual 1,55,223


In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

In [ ]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)